# Feed Forward Neural Network MNIST

### Imports

In [1]:
import torch
import torch.nn as nn
import torchvision.datasets as datasets
import torchvision.transforms as transforms

### Hyperparameters

In [2]:
input_size = 784        # Number of input neurons (image pixels) 28 x 28
hidden_size = 400       # Number of hidden neurons
out_size = 10           # Number of classes (0-9) 
epochs = 10             # How many times we pass our entire dataset into our network 
batch_size = 100        # Input size of the data during one iteration 
learning_rate = 0.001   # How fast we are learning

### Dataset

In [3]:
train_dataset = datasets.MNIST(
    root='./data', 
    train=True, 
    transform=transforms.ToTensor(), 
    download=True
)

test_dataset = datasets.MNIST(
    root='./data',
    train=False,
    transform=transforms.ToTensor()
)

### Data Loader

In [4]:
train_loader = torch.utils.data.DataLoader(
    dataset=train_dataset,
    batch_size=batch_size,
    shuffle=True
)

test_loader = torch.utils.data.DataLoader(
    dataset=test_dataset,
    batch_size=batch_size,
    shuffle=False
)

### Model

In [5]:
class FeedForwardMNIST(nn.Module):
    def __init__(self, input_size, hidden_size, out_size):
        super().__init__()
        self.fc1 = nn.Linear(input_size, hidden_size) # hidden layer 1
        self.fc2 = nn.Linear(hidden_size, hidden_size) # hidden layer 2
        self.fc3 = nn.Linear(hidden_size, out_size) # output layer
        self.relu = nn.ReLU()
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.relu(out)
        out = self.fc3(out)
        return out

### Model Object

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = FeedForwardMNIST(input_size, hidden_size, out_size).to(device)
criteria = nn.CrossEntropyLoss() # Softmax is internally computed so we don't need to add it
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

### Training

In [7]:
for epoch in range(epochs):
    correct = 0
    running_loss = 0.0
    for i, (images, labels) in enumerate(train_loader):
        # Flatten the image and move to GPU if available
        images = images.view(-1, 28*28).to(device)
        labels = labels.to(device)
        # Forward pass
        outputs = model(images)
        # Get predictions from the maximum value
        # It returns the maximum value and the index of the maximum value. We only need the index.
        _, predicted = torch.max(outputs.data, 1)
        correct += (predicted == labels).sum().item()
        # Compute loss
        loss = criteria(outputs, labels)
        running_loss += loss.item()
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f'Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(train_loader):.4f}, Accuracy: {correct/len(train_dataset):.4f}')

Epoch [1/10], Loss: 0.2632, Accuracy: 0.9236
Epoch [2/10], Loss: 0.0968, Accuracy: 0.9706
Epoch [3/10], Loss: 0.0640, Accuracy: 0.9793
Epoch [4/10], Loss: 0.0447, Accuracy: 0.9853
Epoch [5/10], Loss: 0.0333, Accuracy: 0.9893
Epoch [6/10], Loss: 0.0250, Accuracy: 0.9917
Epoch [7/10], Loss: 0.0219, Accuracy: 0.9926
Epoch [8/10], Loss: 0.0180, Accuracy: 0.9940
Epoch [9/10], Loss: 0.0174, Accuracy: 0.9944
Epoch [10/10], Loss: 0.0139, Accuracy: 0.9953


### Testing

In [8]:
with torch.no_grad():
    correct = 0
    for images, labels in test_loader:
        images = images.view(-1, 28*28).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        correct += (predicted == labels).sum().item()
    print(f'Accuracy of the network on the 10000 test images: {correct/len(test_dataset):.4f}')

Accuracy of the network on the 10000 test images: 0.9809
